# Project 1 Flatiron

### Chuck Nadel


In [143]:
# Your code here - remember to use markdown cells for comments as well!

## Importing Packages

In [1]:
import pandas as pd
import sqlite3
import plotly.express as px

## Uploading Databases

In [2]:
# The Movie DB
theMovieDF = pd.read_csv('zippedData/tmdb.movies.csv.gz')

# The Numbers DF
numsDF = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# IMDB SQL Database
conn = sqlite3.connect('zippedData/im.db')
imdbDF = pd.read_sql('''
        SELECT *
        FROM sqlite_master
        WHERE type='table'
        ''', conn)
# IMDB Basics and Ratings Information, merged for ease of analysis
basics_ratingsDF = pd.read_sql('''
        SELECT * FROM movie_basics
        INNER JOIN movie_ratings ON movie_basics.movie_id=movie_ratings.movie_id
        ''', conn)
directors_infoDF = pd.read_sql('''
        SELECT * FROM directors
        INNER JOIN persons ON directors.person_id=persons.person_id
        ''', conn)

In [4]:
# Drop duplicate columns
basics_ratingsDF = basics_ratingsDF.T.drop_duplicates().T
directors_infoDF = directors_infoDF.T.drop_duplicates().T

In [10]:
directors_infoDF = directors_infoDF.drop_duplicates()
complete_imdbDF = pd.merge(basics_ratingsDF, directors_infoDF, on='movie_id', how='left')
#directors_infoDF
#complete_imdbDF
# Lets also drop some columns that we will not be using
complete_imdbDF.drop(['birth_year','death_year','primary_profession'], axis=1, inplace=True)

## Cleaning the Data- GET DIRECTOR

#### First, lets look information provided by each impored Database

In [ ]:
#The Movie Database
theMovieDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


The Movie Dataframe seems to have zero null values in any columns, and the datatypes all make sense, making our job much easier!

In [11]:
complete_imdbDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86783 entries, 0 to 86782
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_id         86783 non-null  object
 1   primary_title    86783 non-null  object
 2   original_title   86783 non-null  object
 3   start_year       86783 non-null  object
 4   runtime_minutes  77588 non-null  object
 5   genres           85844 non-null  object
 6   averagerating    86783 non-null  object
 7   numvotes         86783 non-null  object
 8   person_id        86030 non-null  object
 9   primary_name     86030 non-null  object
dtypes: object(10)
memory usage: 7.3+ MB


Our IMDB dataframe is mostly okay, but it seems that we are missing about 7000 movies' runtime and 800 movies' genres.
In addition, movie_id column repeated itself when merging the tables.
We will address the missing runtime values by finding the median runtime.
Since the number of movies missing a genre is relatively small, we will just drop those rows.

In [ ]:
# The Numbers DF
numsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


At first glance, all seems well here. However, this database does stores the production_budget, domestic_gross, and worldwide_gross columns as objects, as opposed to integers.
To address this, we will use base python to strip the values under these columns such that they can be turned into integers for further analysis

#### Now, lets clean the Basics and Ratings Dataframe and the Numbers Dataframe, based on what we wrote above.

Basics and Ratings Dataframe:

In [14]:
# Dropping Rows without genre
complete_imdbDF.dropna(subset = ['genres'], axis=0, inplace=True)
# Finding Median Runtime
medianRuntime = complete_imdbDF['runtime_minutes'].median()
# Replace Null Values in runtime_minutes with median value
complete_imdbDF.fillna(value = medianRuntime, inplace = True)

In [15]:
complete_imdbDF.isnull().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
person_id          0
primary_name       0
dtype: int64

The Numbers Dataframe:

In [16]:
# Remove $ Sign, commas from production_budget, domestic_gross, and worldwide_gross columns
columnstofix = ['production_budget', 'domestic_gross','worldwide_gross']
for column in columnstofix:
    numsDF[column] = numsDF[column].apply(lambda x:x.replace('$',''))
    numsDF[column] = numsDF[column].apply(lambda x:x.replace(',',''))

In [17]:
# Convert cleaned columns to int data type
for column in columnstofix:
    numsDF[column] = pd.to_numeric(numsDF[column])
# Convert Dates in release_date to datetime object
numsDF['year'] = numsDF['release_date'].str[-4:]
# create new column for 'year'
numsDF['year'] = pd.to_numeric(numsDF['year'], downcast='integer')

Create a new column; international profit

In [18]:
numsDF['Worldwide_Profit'] = numsDF['worldwide_gross']-numsDF['production_budget']
numsDF.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,Worldwide_Profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,999721747


### #1 Safest (profit-wise) Movies by Genre

Let's look at the % of movies in the green, black, and red based on their Genre

Green = 100%+ Profit

Black = 0-100% Profit 

Red = Lost Money

In [19]:
# First, let's create a new numerical column in NumsDF, %ROI (Profit/Budget)
numsDF['% Return'] = numsDF['Worldwide_Profit']/numsDF['production_budget']*100
# Now, let's categorize these results in a new column, simple_success, based on the parameters above
numsDF['simple_success'] = numsDF['% Return'].apply(lambda x: 'Yes' if x >= 100 else 'No' if x < 0 else 'Breakeven')
# Finally, we need to merge this dataframe with the genre series in the IMDB dataframe
numsimdbDF = complete_imdbDF.merge(numsDF, how = 'inner', left_on = ['primary_title', 'start_year'], right_on = ['movie', 'year'])
# Since the genres listed together, we want to make a dataframe where their is only 1 genre per column. We will do this by splitting the genre entries, and then copying each column over
# Split by genres
numsimdbDF['genres'] = numsimdbDF['genres'].str.split(',')
numsimdbDF
genre_list = [{'Profit?':row.simple_success,'genres':g} for idx, row in numsimdbDF.iterrows() for g in row.genres ]
# Convert it to dataframe
profit_genreDF = pd.DataFrame(genre_list)

In [20]:
grouped_df = profit_genreDF.groupby('genres')
profit_genreDF['Profit?'].value_counts().sort_values(ascending=False)

Yes          2263
No           1141
Breakeven     765
Name: Profit?, dtype: int64

In [22]:
df_stacked = profit_genreDF.groupby(['genres', 'Profit?']).size().reset_index(name='counts')
df_stacked['counts'] = df_stacked.groupby('genres')['counts'].apply(lambda x: x/x.sum())
df_stacked.head()
df_green = df_stacked[df_stacked['Profit?'] == 'Yes']
df_stacked.head(20)
genreProfitStacked = px.bar(df_stacked, x = 'genres', y = 'counts', color = 'Profit?', barmode = 'stack', color_discrete_map = {'Breakeven': 'black', 'Yes':'seagreen','No':'darkred'})
genreProfitStacked.update_layout(yaxis_title='Movie Profitability Rate', xaxis_title='Genres')

In [23]:
genreProfitGreen = px.bar(df_green, x = 'genres', y = 'counts', color = 'Profit?', barmode = 'stack', color_discrete_map = {'Yes':'seagreen'}, range_y = [0,1])
genreProfitGreen.update_layout(yaxis_title='Green Percentage', xaxis_title='Genres')

### #2 Correlation between number of IMDB Reviews, Box Office Profit

In [28]:
# We will use the originally merged database, numsimdbDF for this analysis
numsimdbDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1652 entries, 0 to 1651
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           1652 non-null   object 
 1   primary_title      1652 non-null   object 
 2   original_title     1652 non-null   object 
 3   start_year         1652 non-null   int64  
 4   runtime_minutes    1652 non-null   float64
 5   genres             1652 non-null   object 
 6   averagerating      1652 non-null   float64
 7   numvotes           1652 non-null   int64  
 8   person_id          1652 non-null   object 
 9   primary_name       1652 non-null   object 
 10  id                 1652 non-null   int64  
 11  release_date       1652 non-null   object 
 12  movie              1652 non-null   object 
 13  production_budget  1652 non-null   int64  
 14  domestic_gross     1652 non-null   int64  
 15  worldwide_gross    1652 non-null   int64  
 16  year               1652 

In [25]:
# Let's make a new DF with primary_title, numvotes, Worldwide_Profit, and % Return
question2DF = numsimdbDF[['primary_title','numvotes','Worldwide_Profit', '% Return']]
question2DF.head()

,primary_title,numvotes,Worldwide_Profit,% Return
0,Foodfight!,8248,-44926294,-99.836209
1,The Secret Life of Walter Mitty,275300,96861183,106.440860
2,A Walk Among the Tombstones,105116,34108587,121.816382
3,Jurassic World,539338,1433854864,666.909239
4,The Rum Diary,94787,-23455268,-52.122818


Lets graph numvotes and Worldwide_Profit on a scatterplot

In [26]:
votesProfit = px.scatter(question2DF, x="numvotes", y="Worldwide_Profit", trendline = 'ols')
correlation = question2DF["numvotes"].corr(question2DF["Worldwide_Profit"])
votesProfit.update_layout(annotations=[dict(x=0.95, y=0.95, xref='paper', yref='paper', text=f'correlation: {correlation:.2f}', 
                                 showarrow=False, font=dict(size=14))],xaxis_title='Number of Votes', yaxis_title='Worldwide Profit')

It appears that their is a positive correlation between the number of votes a film recieves on IMDB, and the profit it made. Let's explore this more by finding the correlation coefficient
Look into directors that get most votes and ROI, find intersect.

In [27]:
question2DF["numvotes"].corr(question2DF["Worldwide_Profit"])

0.6201159136180605

Now how about we look at Profit % versus numvotes

In [29]:
votesProfit = px.scatter(question2DF, x="numvotes", y="% Return")
correlation = question2DF["numvotes"].corr(question2DF["% Return"])
votesProfit.update_layout(annotations=[dict(x=0.95, y=0.95, xref='paper', yref='paper', text=f'correlation: {correlation:.2f}', 
                                 showarrow=False, font=dict(size=14))],xaxis_title='Number of Votes', yaxis_title='% Return')

So, their is no correlation between the number of votes a movie recieves and its % Return.

Let's now explore which directors generate the highest number of votes 

In [30]:
complete_imdbDF.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,person_id,primary_name
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,nm0712540,Harnam Singh Rawail
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,nm0002411,Mani Kaul
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,nm0000080,Orson Welles
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,90.0,"Comedy,Drama",6.1,13,nm0611531,Hrishikesh Mukherjee
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,nm0765384,Valeria Sarmiento


In [31]:
numsDF

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,Worldwide_Profit,% Return,simple_success
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2009,2351345279,553.257713,Yes
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,635063875,154.667286,Yes
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,2019,-200237650,-57.210757,No
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,2015,1072413963,324.384139,Yes
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,999721747,315.369636,Yes
...,...,...,...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,2018,-7000,-100.000000,No
5778,79,"Apr 2, 1999",Following,6000,48482,240495,1999,234495,3908.250000,Yes
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,2005,-3662,-73.240000,No
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,2015,-1400,-100.000000,No


In [82]:
# Let's make another dataframe with directors, their movies, and the number of imdb votes recieved
completeDF = complete_imdbDF.merge(numsDF, how = 'inner', left_on = ['primary_title', 'start_year'], right_on = ['movie', 'year'])
directors_votesDF = completeDF[['primary_name','numvotes', 'Worldwide_Profit']]
#completeDF = complete_imdbDF.merge(numsDF, how = 'inner', left_on = ['primary_title', 'start_year'], right_on = ['movie', 'year'])
directors_votesDF = directors_votesDF.groupby(['primary_name']).sum().sort_values(by=['numvotes'], ascending=False).reset_index()
directors_votesDF = directors_votesDF.head(50)
directors_votesCorr = directors_votesDF["numvotes"].corr(question2DF["Worldwide_Profit"])
directors_votesTopTen = directors_votesDF.head(10)
directors_votesTopTen

,primary_name,numvotes,Worldwide_Profit
0,Christopher Nolan,4994749,2336180484
1,Martin Scorsese,2411429,486606606
2,Ridley Scott,1928190,1253023808
3,Anthony Russo,1920685,3182605502
4,Joe Russo,1920685,3182605502
5,Denis Villeneuve,1870701,361519696
6,Joss Whedon,1849249,2365349860
7,Zack Snyder,1845493,1470920114
8,Peter Jackson,1723813,2172948044
9,David Fincher,1717750,641863294


In [68]:
px.scatter(directors_votesDF, x='numvotes', y='Worldwide_Profit',hover_name='primary_name')

Christopher Nolan is the best. Joe Russo and Pierre Coffin are also good but they also have partners

### #3 Budget vs Return on Investment

#1 Normal Budget vs ROI

#2 Budget vs Worldwide_Profit

#3 By Director

In [61]:
Question3DF = completeDF[['primary_title','production_budget','% Return','primary_name', 'Worldwide_Profit']]
Question3DF

,primary_title,production_budget,% Return,primary_name,Worldwide_Profit
0,Foodfight!,45000000,-99.836209,Lawrence Kasanoff,-44926294
1,The Secret Life of Walter Mitty,91000000,106.440860,Ben Stiller,96861183
2,A Walk Among the Tombstones,28000000,121.816382,Scott Frank,34108587
3,Jurassic World,215000000,666.909239,Colin Trevorrow,1433854864
4,The Rum Diary,45000000,-52.122818,Bruce Robinson,-23455268
...,...,...,...,...,...
1647,Happy Death Day 2U,9000000,613.105500,Christopher Landon,55179495
1648,Blinded by the Light,15000000,-100.000000,Gurinder Chadha,-15000000
1649,Fahrenheit 11/9,5000000,33.074300,Michael Moore,1653715
1650,Unplanned,6000000,201.793683,Chuck Konzelman,12107621


In [62]:
# It appears we have an some outliers ruining our graph. Let's drop those entries
Question3DF.drop(Question3DF.loc[Question3DF['primary_title'] == 'The Gallows'].index, inplace= True)
Question3DF.drop(Question3DF.loc[Question3DF['primary_title'] == 'The Devil Inside'].index, inplace= True)
px.scatter(Question3DF, x = 'production_budget', y = '% Return',hover_name='primary_title')

c:\Users\cnade\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
# Not much there. Let's look at budget vs profit
BudgetProfit = px.scatter(Question3DF, x = 'production_budget', y = 'Worldwide_Profit',hover_name='primary_title')
# Correlation Coefficient
correlationQ3 = Question3DF["production_budget"].corr(Question3DF["Worldwide_Profit"])
correlationQ3
# Hey, thats pretty good! Let's update our graph with that correlation coefficient
BudgetProfit.update_layout(annotations=[dict(x=0.95, y=0.95, xref='paper', yref='paper', text=f'correlation: {correlationQ3:.3f}', 
                                 showarrow=False, font=dict(size=14))],xaxis_title='Number of Votes', yaxis_title='% Return')

In [77]:
# By Director
BudProfDirectorDF = Question3DF[['primary_name','production_budget','Worldwide_Profit']]
BudProfDirectorDF = BudProfDirectorDF.groupby(['primary_name']).sum().sort_values(by=['Worldwide_Profit'], ascending=False).reset_index()
BudProfDirectorCorr = BudProfDirectorDF['production_budget'].corr(BudProfDirectorDF['Worldwide_Profit'])
BudProfDirectorCorr
BudProfDirector = px.scatter(Question3DF, x = 'production_budget', y = 'Worldwide_Profit', hover_name='primary_name')
BudProfDirector.update_layout(annotations=[dict(x=0.95, y=0.95, xref='paper', yref='paper', text=f'correlation: {BudProfDirectorCorr:.3f}', 
                                 showarrow=False, font=dict(size=14))],xaxis_title='Production Budget', yaxis_title='Worldwide Profit')
BudProfDirector